In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/Token'

files = os.listdir(folder_path)
csv_files = [file for file in files if file.endswith('.csv')]
for file in csv_files:
    print(file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
rt_processed.csv
bbc_processed.csv
cnn_processed.csv
guardian_processed.csv
combined_processed_data.csv


In [ ]:
import pandas as pd
import re

In [ ]:
processed_data = pd.read_csv('/content/drive/MyDrive/Token/' + 'combined_processed_data.csv')

In [ ]:
processed_data.shape

(68587, 7)

In [ ]:
processed_data.source

,source
0,bbc
1,bbc
2,bbc
3,bbc
4,bbc
...,...
68582,rt
68583,rt
68584,rt
68585,rt


In [ ]:
a = list(processed_data.source)

In [ ]:
for k in ['bbc', 'cnn', 'guardian', 'rt']:
  if k in a:
    print(k)

bbc
cnn
guardian
rt


In [ ]:
tokens = {}
for file in csv_files:
  news_origin = re.sub('(.*)_processed.csv', r'\1', file)
  token = pd.read_csv('/content/drive/MyDrive/Token/' + file)
  tokens[news_origin] = token

In [ ]:
for key, value in tokens.items():
  print(value.shape)
  print(value.columns)

(9800, 10)
Index(['id', 'indexed_date', 'language', 'media_name', 'media_url',
       'publish_date', 'title', 'url', 'body', 'processed_body'],
      dtype='object')
(7601, 7)
Index(['language', 'media_name', 'publish_date', 'title', 'url', 'body',
       'processed_body'],
      dtype='object')
(34294, 7)
Index(['id', 'title', 'url', 'published_date', 'body', 'table',
       'processed_body'],
      dtype='object')
(16892, 6)
Index(['url', 'publish_date', 'title', 'text', 'language', 'processed_body'], dtype='object')


In [ ]:
df0 = {}

In [ ]:
df0['bbc'] = tokens['bbc'][['url', 'publish_date', 'title', 'body', 'language', 'processed_body']]
df0['cnn'] = tokens['cnn'][['url', 'publish_date', 'title', 'body', 'language', 'processed_body']]
tokens['guardian'].columns = tokens['guardian'].columns.str.replace('source_', '', regex=True)
df0['guardian'] = tokens['guardian'][['url', 'published_date', 'title', 'body',  'processed_body']]
df0['rt'] = tokens['rt'].rename(columns={'text': 'body'})
df0['guardian'] = df0['guardian'].rename(columns={'published_date': 'publish_date'})
df0['guardian']['language'] = 'en'

In [ ]:
for key, value in df0.items():
  print(value.shape)
  print(value.columns)

(9800, 7)
Index(['url', 'publish_date', 'title', 'body', 'language', 'processed_body',
       'source'],
      dtype='object')
(7601, 7)
Index(['url', 'publish_date', 'title', 'body', 'language', 'processed_body',
       'source'],
      dtype='object')
(34294, 7)
Index(['url', 'publish_date', 'title', 'body', 'processed_body', 'language',
       'source'],
      dtype='object')
(16892, 7)
Index(['url', 'publish_date', 'title', 'body', 'language', 'processed_body',
       'source'],
      dtype='object')


In [ ]:
for key, value in df0.items():
  df0[key]['publish_date'] = pd.to_datetime(df0[key]['publish_date']).dt.date
  print(df0[key]['publish_date'][0])

2024-10-03
2024-11-15
2022-04-12
2024-11-07


<ipython-input-102-1243c42ccf7e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df0[key]['publish_date'] = pd.to_datetime(df0[key]['publish_date']).dt.date
<ipython-input-102-1243c42ccf7e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df0[key]['publish_date'] = pd.to_datetime(df0[key]['publish_date']).dt.date


In [ ]:
df0['rt']['processed_body'][0]

"['Sitting', 'artificial', 'intelligence', 'development', 'race', 'option', 'Russia', 'President', 'Vladimir', 'Putin', 'said', 'Putin', 'commented', 'technology', 'response', 'question', 'Thursdays', 'plenary', 'session', 'Valdai', 'Discussion', 'Forum', 'annual', 'event', 'Russian', 'city', 'Sochi', 'I', 'think', 'impossible', 'ban', 'It', 'find', 'way', 'especially', 'competition', 'competition', 'growing', 'Putin', 'told', 'audience', 'Moscow', 'inclined', 'regulate', 'AI', 'instead', 'trying', 'prohibit', 'use', 'said', 'The', 'development', 'AI', 'inevitable', 'among', 'leaders', 'given', 'advantages', 'Russian', 'president', 'added', 'Leading', 'development', 'AI', 'online', 'platforms', 'founded', 'West', 'tendency', 'shape', 'views', 'users', 'Putin', 'noted', 'We', 'need', 'understand', 'develop', 'sovereign', 'AI', 'using', 'everything', 'Russian', 'president', 'said', 'Putin', 'compared', 'AI', 'debate', 'upsides', 'downsides', 'nuclear', 'age', 'Using', 'atom', 'peacefully

In [ ]:
def remove_punctuation(tokens):
    return [token for token in tokens if not re.match(r'[^\w\s]', token)]

for key, value in df0.items():
    print(f"Processing {key}...")

    value['processed_body'] = value['processed_body'].apply(eval)  # 如果它是字符串形式的列表，转为真正的列表

    value['processed_body'] = value['processed_body'].apply(remove_punctuation)

    print(f"Processed {key} shape: {value.shape}")
    print(f"Processed {key} columns: {value.columns}")

In [ ]:
token_list = []
for key, value in df0.items():
  value['source'] = key
  token_list.append(value)

<ipython-input-122-91d272265fa1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value['source'] = key


In [ ]:
df = pd.concat(token_list)

In [ ]:
df.to_csv("/content/drive/MyDrive/Token/combined_processed_data.csv", index=False)

In [ ]:
df.sample(20)

,url,publish_date,title,body,language,processed_body,source
17306,https://www.theguardian.com/politics/2024/mar/...,2024-03-04,Galloway win makes Workers party a focus for f...,"Until George GallowaysRochdale byelection win,...",en,"[Until, George, GallowaysRochdale, byelection,...",guardian
8849,https://www.bbc.co.uk/news/entertainment-arts-...,2022-03-13,Bafta Film Awards 2022 red carpet in pictures,Bafta Film Awards 2022 red carpet in pictures ...,en,"[Bafta, Film, Awards, 2022, red, carpet, pictu...",bbc
4619,https://www.theguardian.com/news/2023/jan/20/c...,2023-01-20,Corrections and clarifications,"Gaia Vinces latest book is Nomad Century, not ...",en,"[Gaia, Vinces, latest, book, Nomad, Century, T...",guardian
34010,https://www.theguardian.com/world/live/2024/au...,2024-08-02,Middle East crisis: assassinated Hamas leader ...,3.45pmBSTClosing summaryMourners gathered in D...,en,"[3.45pmBSTClosing, summaryMourners, gathered, ...",guardian
9695,https://www.bbc.co.uk/news/uk-scotland-6050440...,2022-02-24,Scotland's papers: Salmond under fire and Kyiv...,Scotland's papers: Salmond under fire and Kyiv...,en,"[Scotland, papers, Salmond, fire, Kyiv, state,...",bbc
22846,https://www.theguardian.com/world/2022/sep/20/...,2022-09-20,‘We thieves and killers are now fighting Russi...,"The inmates of penal colony No 8, in the Tambo...",en,"[The, inmates, penal, colony, No, 8, Tambov, r...",guardian
4139,https://www.rt.com/russia/595712-sushentsov-in...,2024-04-10,Andrey Sushentsov: Americans can’t tell us who...,The United States of America is trying to cont...,en,"[The, United, States, America, trying, control...",rt
3481,https://www.rt.com/russia/597496-hunka-canada-...,2024-05-13,Canada ‘whitewashing’ Nazi crimes – Russian pr...,Ottawas failure to prosecute a former Nazi sol...,en,"[Ottawas, failure, prosecute, former, Nazi, so...",rt
4796,https://www.rt.com/africa/594132-africa-us-bud...,2024-03-12,US State Dept requesting funds to counter Russ...,The US State Department has requested $322.5 m...,en,"[The, US, State, Department, requested, 322.5,...",rt
5924,https://www.rt.com/news/591072-us-ukraine-russ...,2024-01-22,US forming ‘colonial administration’ in Ukrain...,Washington has begun creating a colonial admin...,en,"[Washington, begun, creating, colonial, admini...",rt


In [ ]:
df.shape

(68587, 6)